In [ ]:
!cp ../input/hashcode-drone-delivery/busy_day.in a.in

In [ ]:
%%writefile main.cpp
// #include <bits/stdc++.h>
#include <algorithm>
#include <vector>
#include <string>
#include <unordered_map>
#include <iostream>
#include <fstream>
#include <time.h>
#include <bitset>
#include <cmath>
#include <queue>   
#include <deque>
#include <assert.h>

using namespace std;

typedef vector<int> vi;
typedef vector<vi> vvi;

typedef pair<int, int> pi;
typedef vector<pair<int, int >> vp;



template <typename Type>
vector<vector<Type>> create_multivector(int dim1, int dim2, Type init_data = 0) {
    return vector<vector<Type>> (dim1, vector<Type> (dim2, init_data));
}



// environment params
int nRows, nCols, nTypes, nWarehouses, nOrders, nDrones, Time, MAX_LOAD, n_completed;
int total_score = 0;

vi type_weights;




struct Warehouse {
    int row;
    int col;

    vi available_products;

    Warehouse(int r, int c) : row(r), col(c) {};

};

enum CmdType
{
    LOAD,
    DELIVER,
    UNLOAD,
    WAIT,
    NONE
};


struct Command {
    // common params
    CmdType type;
    int drone_id;
    int start_time;
    int end_time;

    // for load/unload
    int warehouse_id;
    int product_type;

    // for deliver
    int order_id;
    int customer_id;

    int num_items; // to be loaded or unloaded or delivered
    
};


struct Order {
    int row, col; // destination
    int L; // num of ordered product items
    int order_id;

    bool completed = false;

    vi prod_types;
    vi prod_idx;

    Order(int r, int c, int l, int i) : row(r), col(c), L(l), order_id{i} {};
};


class Drone;
vector<Warehouse> Warehouses;
vector<Order> Orders;
vector<Drone> Drones;



// vector<Warehouse> FakeWarehouses;
void refresh_fake_warehouses(vector<Warehouse>& fake_w)
{
    for (int i = 0; i < Warehouses.size(); ++i)
    {
        Warehouses[i] = fake_w[i];
        // & true_w = Warehouses[i];
        // Warehouse& fake_w = FakeWarehouses[i];
        // fake_w.row = true_w.row;
        // fake_w.col = true_w.col;
        // for (int j = 0; j < fake_w.available_products.size(); ++j) {         
    }
}



int completion_time(const Order& order, const Drone& drone);

class Drone {
public:
    int row, col; // current position
    bool busy; // if it is doing something rn
    int drone_id;

    vector<int> prod_types;

    int busy_until = 0;
    vector<Command> commands;


    Drone(int id, int row, int col)
    {
        drone_id = id;
        this->row = row;
        this->col = col;
        for (int i = 0; i < nTypes; ++i)
            prod_types.push_back(0);
    }


    Drone(const Drone& d): row(d.row), col(d.col), busy(d.busy), drone_id(d.drone_id), busy_until(d.busy_until), prod_types(d.prod_types)  {
/*
       for (auto type: d.prod_types) {
           prod_types.push_back(type);
       }
       cout << "WTF2222??" << endl;*/
    };


/*
    Drone(const Drone& d) {
cout << "WTF2222??" << endl;
}
*/


    // int find_task() {
        
        
    //     // busy_until = completion_time()
    //     // return busy_until;
    // }

    // bool is_full()
    // {
        
    // }
};




int calc_dist(int r1, int c1, int r2, int c2)
{
    return ceil(sqrt(double(r1 - r2) * double(r1 - r2) + double(c1 - c2) * double(c1 - c2)));
}


bool need_this_warehouse(Order& order, Drone& drone, Warehouse& ware, int load) {
    for (int type = 0; type < nTypes; type++) {
        // cout << "Type " << type << " need " << order.prod_types[type] << " have " << ware.available_products[type] << endl;
        
        if (order.prod_types[type] > 0 && \
            load + type_weights[type] <= MAX_LOAD && \
            ware.available_products[type] > 0) {
            return true;
        }
    }
    // cout << endl;
    // cout << "NOT TRUEEEE" << endl;

    return false;
}


vector<Command> ALL_FUCKING_COMMANDS;


int completion_time(Order& true_order, Drone& true_drone, int start_time, bool inplace)
{
    int current_load = 0;
    Drone drone(true_drone); // make a copy
    Order order(true_order);
    vector<Warehouse> fake_warehouses;
    for (int i = 0; i < nWarehouses; ++i)
    {
        fake_warehouses.push_back(Warehouse(Warehouses[i]));
    }

    int drone_time = start_time;

    vector<Command> commands;


    // cout << drone.row << " " << drone.col << endl;
    // cout << true_drone.row << " " << true_drone.col << endl;

    // cout << endl << endl << "START LOOK AT SOME ORDER. this order: " << endl;
    // for (int type : order.prod_idx) {
    //     cout << order.prod_types[type] << " ";
    // }

    // cout << endl;


    while (true)
    {

        int best_wrhs_i = -1, shortest_dist = 1000000000;
        for (int i = 0; i < fake_warehouses.size(); ++i) 
        {
            int dist = calc_dist(
                fake_warehouses[i].row, fake_warehouses[i].col, 
                drone.row, drone.col);
            // cout << "CALC DIST " << drone.row << " " << drone.col << " " << dist << endl;
            if (dist < shortest_dist && need_this_warehouse(order, drone, fake_warehouses[i], current_load)) {
                shortest_dist = dist;
                best_wrhs_i = i;
            }
        }

        // cout << "Best warehouse " << best_wrhs_i << endl;

        if (best_wrhs_i == -1) {
            // deliver to drop and repeat
            for (int type : order.prod_idx) {
                // cout << "Need " << order.prod_types[type] << " of " << type << endl;
                assert(order.prod_types[type] >= 0);
                assert(order.prod_types[type] <= 1000);
                if (drone.prod_types[type]) {
                    // unload    
                    int unload_amount = (int) drone.prod_types[type];

                    Command temp_com;
                    temp_com.type = DELIVER;
                    temp_com.drone_id = drone.drone_id;
                    temp_com.order_id = order.order_id;
                    temp_com.product_type = type;
                    temp_com.num_items = unload_amount;
                    temp_com.start_time = drone_time;

                    drone.prod_types[type] -= unload_amount;
                    current_load -= unload_amount * type_weights[type];

                    if (drone.col == order.col && drone.row == order.row) {
                        drone_time++;
                    } else {
                        drone_time += calc_dist(order.row, order.col, drone.row, drone.col) + 1;
                        drone.col = order.col;
                        drone.row = order.row;
                    }
                    temp_com.end_time = drone_time;

                    commands.push_back(temp_com);
                    // cout << "Finished Delivering" << endl;
                }

            }

            assert(current_load == 0);

            

            bool cont = false;
            for (int type : order.prod_idx) {
                if (order.prod_types[type]) {
                    cont = true;
                }
            }
            
            if (!cont) {
                break;
                cout << "ORDER CAN BE COMPLETED! ENDING" << endl;
                // order completed
            }
            continue;
        }
        // fly to this warehouse and load

        // what to load

        Warehouse& best_warehouse = fake_warehouses[best_wrhs_i];

        for (int type : order.prod_idx) {
            
            // cout << "Need " << order.prod_types[type] << " of " << type << endl;
            assert(order.prod_types[type] >= 0);
            assert(order.prod_types[type] <= 20);
            if (order.prod_types[type] && \
                best_warehouse.available_products[type]) {
                // load    
                int temp_amount = min((int) ((MAX_LOAD - current_load) / type_weights[type]), order.prod_types[type]);

                temp_amount = min(temp_amount, best_warehouse.available_products[type]);


                if (!temp_amount) {
                    continue;
                }

                // cout << "Go and load this much: " << temp_amount  << endl;

                Command temp_com;
                temp_com.type = LOAD;
                temp_com.drone_id = drone.drone_id;
                temp_com.warehouse_id = best_wrhs_i;
                temp_com.product_type = type;
                temp_com.num_items = temp_amount;
                temp_com.start_time = drone_time;

                // update the prod_types load and current_load
                drone.prod_types[type] += temp_amount;
                current_load += temp_amount * type_weights[type];
                order.prod_types[type] -= temp_amount;
                best_warehouse.available_products[type] -= temp_amount;

                if (drone.col == best_warehouse.col && drone.row == best_warehouse.row) {
                    drone_time++;
                } else {
                    drone_time += shortest_dist + 1;
                    drone.col = best_warehouse.col;
                    drone.row = best_warehouse.row;
                }
                temp_com.end_time = drone_time;

                commands.push_back(temp_com);
            }

        }       
    }


    if (inplace) {
        true_drone = drone;
        true_order = order;
        true_order.completed = true;
        n_completed++;
        refresh_fake_warehouses(fake_warehouses);
        
        if (drone_time < Time) {
            for (auto command : commands) {
                ALL_FUCKING_COMMANDS.push_back(command);
                // true_drone.commands.push_back(command);
            }
            total_score += ceil(100*(double(Time) - double(drone_time)) / double(Time));
        }
    }


    return drone_time;
    
}


class PairCompare
{
public:
    bool operator() (pair<int,int> a, pair<int,int> b)
    {
        return a.first > b.first;
    }
};


int main(int argc, char* argv[]) {
    clock_t ttime;
    ttime = clock();
    ios_base::sync_with_stdio(false);
    cin.tie(0);
    cout.tie(0);
    vector<string> filenames {"a", \
                            "b", \
                            "c",
                            "d"};

    int file_index = 0;

    if (argc >= 2) {
        file_index = atoi(argv[1]);
    }


    ifstream infile(filenames[file_index] + ".in");
    if (!infile) {
        cerr << "Cannot open infile!\n";
        return 0;
    }

    //
    // Reading input -- use infile exactly as you use std::cin
    //

    int N, M;

    infile >> nRows >> nCols >> nDrones >> Time >> MAX_LOAD;

    infile >> nTypes;


    for (int i = 0; i < nTypes; i++) {
        int temp;
        infile >> temp;
        type_weights.push_back(temp);
    }


    infile >> nWarehouses;

    for (int i = 0; i < nWarehouses; i++) {
        int t_row, t_col;
        infile >> t_row >> t_col;
        cout << "Read " << t_row << " " << t_col << endl;
        Warehouse temp_w(t_row, t_col);
        
        vi temp_v;
        
        for (int j = 0; j < nTypes; j++) {
            int temp_n;
            infile >> temp_n;
            temp_v.push_back(temp_n);
        }

        temp_w.available_products = temp_v;
        Warehouses.push_back(temp_w);
        // FakeWarehouses.push_back(Warehouse(temp_w));
    }


    cout << "Start " << Warehouses[0].row << " " << Warehouses[0].col << endl;

    infile >> nOrders;

    for (int i = 0; i < nOrders; i++) {
        int t_row, t_col, t_L;
        infile >> t_row >> t_col >> t_L;
        Order temp_order(t_row, t_col, t_L, i);
        
        temp_order.prod_types = vector<int>(nTypes, 0);
        

        for (int j = 0; j < t_L; j++) {
            int temp_n;
            infile >> temp_n;
            temp_order.prod_idx.push_back(temp_n);
            temp_order.prod_types[temp_n]++;
        }


        /*
        cout << endl <<  "Order " << i << " " << endl;
        for (int type = 0; type < nTypes; type++) {
            cout << temp_order.prod_types[type] << " ";
        }
        */
        Orders.push_back(temp_order);
    }


    /*
    for (int i = 0; i < nOrders; i++) {
        cout << endl <<  "Order " << i << " " << endl;
        for (int type = 0; type < nTypes; type++) {
            cout << Orders[i].prod_types[type] << " ";
        }
    }

    cout << endl;
    */
    //
    //
    infile.close();


    for (int i = 0; i < nDrones; ++i)
    {
        Drones.push_back(Drone(i, Warehouses[0].row, Warehouses[0].col));
    }



    cout << Drones[0].row << " " << Drones[0].col << endl;
   

    int t = 0;

    auto comp = [](pair<int, int> a, pair<int, int> b) {
        return (a.first > b.first);
    };
    priority_queue<pair<int, int>, vector<pair<int, int>>, PairCompare> q;

    for (int i = 0; i < nDrones; i++) {
        q.push({0, i});
    }

    // cout << nDrones << endl;

    n_completed = 0;

    cout << "AT BEGIN:" << endl;


    for (int ware_id = 0; ware_id < nWarehouses; ware_id++) {
        cout << "At warehouse " << ware_id << endl;
        for (int type = 0; type < nTypes; type++) {
            cout << Warehouses[ware_id].available_products[type] << " of type " << type << endl;
        }
    }
    cout << endl << endl;


    while (t < Time && n_completed < nOrders) {
        pi curr_elem = q.top();
        q.pop();
        t = curr_elem.first;
        int curr_drone = curr_elem.second;
        cout << "Current drone " << curr_drone << endl;
        cout << "Current time " << t << endl;
        
        cout << "Current pos " << Drones[curr_drone].row << " " <<  Drones[curr_drone].col << endl;
        // int next_time = Drones[curr_drone].find_task();
        
        int best_time = 1000000000;
        int best_id = -1;

        cout << "Number of remaining orders before " << nOrders - n_completed << endl;

        for (int n_order = 0; n_order < nOrders; n_order++) {
            if (Orders[n_order].completed) {
                continue;
            }
            // cout << "ADADADAD" << endl;
            int this_time = completion_time(Orders[n_order], Drones[curr_drone], t, false);
            if (this_time < best_time) {
                best_time = this_time;
                best_id = n_order;
            }
        }
        //if (best_time < Time) {
            int next_time = completion_time(Orders[best_id], Drones[curr_drone], t, true);
        //}
        assert(Orders[best_id].completed == true);

        cout << "Time " << t << ", number of remaining orders after " << nOrders - n_completed << endl;

        

        q.push({next_time, curr_drone});
    }


    cout << "Exited Cycle" << endl;

    cout << "AT END:" << endl;


    for (int ware_id = 0; ware_id < nWarehouses; ware_id++) {
        cout << "At warehouse " << ware_id << endl;
        for (int type = 0; type < nTypes; type++) {
            cout << Warehouses[ware_id].available_products[type] << " of type " << type << endl;
        }
    }
    cout << endl << endl;



    //
    // for progress tracking
    //

    clock_t ttime1;
    ttime1 = clock();

    // int parameter_to_track;

    // int tracking_step = 200;
    // if (argc >= 3) {
    //     tracking_step = atoi(argv[2]);
    // }

    // while (true) {
    //     if (!(parameter_to_track % tracking_step)) {
    //         cout << ((float) (clock() - ttime1) )/CLOCKS_PER_SEC \
    //             << " seconds for the last " << tracking_step \
    //             << " iterations. Current number is " << parameter_to_track << endl;
    //         ttime1 = clock();
    //     }
    // }


    //
    //
    //


    cout << "Total score " << total_score << endl;

    ttime = clock() - ttime;
    cout << "Time " << ((float) ttime )/CLOCKS_PER_SEC << "\n";


    ofstream outfile(filenames[file_index] + ".out");

    if (!outfile) {
        cout << "Cannot open outfile!\n";
        return 0;
    }




    //
    // Writing output -- use outfile exactly as you use std::cout
    //

    // int total_commands = 0;
    // for (auto& drone : Drones) {
    //     total_commands += drone.commands.size();
    // }



    // outfile << total_commands << endl;
    outfile << ALL_FUCKING_COMMANDS.size() << endl;


    // for (auto& drone : Drones) {
    //     for (auto& com : drone.commands) {
    //         outfile << drone.drone_id << " ";
    //         if (com.type == LOAD) {
    //             outfile << "L" << " ";
    //             outfile << com.warehouse_id << " " << com.product_type << " " << com.num_items << endl;
    //         } else {
    //             outfile << "D" << " ";
    //             outfile << com.order_id << " " << com.product_type << " " << com.num_items << endl;
    //         }
    //     }
    // }
    for (auto& com : ALL_FUCKING_COMMANDS) {
        outfile << com.drone_id << " ";
        if (com.type == LOAD) {
            outfile << "L" << " ";
            outfile << com.warehouse_id << " " << com.product_type << " " << com.num_items << endl;
        } else {
            outfile << "D" << " ";
            outfile << com.order_id << " " << com.product_type << " " << com.num_items << endl;
        }
    }




    outfile.close();
    return 0;
}



In [ ]:
!g++ -std=c++11 main.cpp -o main

In [ ]:
!./main

In [ ]:
!cp a.out submission.csv